In [1]:
#from Adapt import AdaptVQE_Y

from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureMoleculeDriver, ElectronicStructureDriverType
from qiskit_nature.mappers.second_quantization import ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer
import warnings
from qiskit_nature.circuit.library import HartreeFock,UCCSD
from qiskit import Aer
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP

from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator

from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE

from qiskit_nature.settings import QiskitNatureSettings
QiskitNatureSettings.dict_aux_operators = True

import numpy as np 
warnings.filterwarnings("ignore")

/tmp/ipykernel_27912/1528283346.py:17: NatureDeprecationWarning: The qiskit_nature.algorithms.excited_state_solvers package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.excited_state_solvers package.
  from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
/tmp/ipykernel_27912/1528283346.py:17: NatureDeprecationWarning: The qiskit_nature.algorithms.pes_sampler package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release . Please refer to 'https://github.com/Qiskit/qiskit-nature/issues/750' for additional information regarding 'extrapolators'.
  from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory


In [2]:
molecule = Molecule(
    # coordinates are given in Angstrom
    geometry=[
        ["O", [0.0, 0.0, 0.0]],
        ["H", [0.758602, 0.0, 0.504284]],
        ["H", [0.758602, 0.0, -0.504284]]
    ],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)

driver = ElectronicStructureMoleculeDriver(
    molecule=molecule,
    basis="sto3g",
    driver_type=ElectronicStructureDriverType.PYSCF,
)

#driver = PySCFDriver(atom="O 0 0 0;H 0.758602 0 0.504284;H 0.758602 0 -0.504284",basis="sto-3g")

properties = driver.run()
converter = QubitConverter(JordanWignerMapper())
PN_property = properties.get_property("ParticleNumber")
print(PN_property)

# Define the active space around the Fermi level 
# (selected automatically around the HOMO and LUMO, ordered by energy)
transformer = ActiveSpaceTransformer(
    num_electrons=2, #how many electrons we have in our active space
    num_molecular_orbitals=2, #how many orbitals we have in our active space
)

problem = ElectronicStructureProblem(driver, [transformer])
second_q_ops = problem.second_q_ops()  # Get 2nd Quant OP
num_spin_orbitals = problem.num_spin_orbitals
num_particles = problem.num_particles
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
print('轨道数目：{a}\n粒子数目{b}\n'.format(a=num_spin_orbitals,b=num_particles))


ParticleNumber:
	14 SOs
	5 alpha electrons
		orbital occupation: [1. 1. 1. 1. 1. 0. 0.]
	5 beta electrons
		orbital occupation: [1. 1. 1. 1. 1. 0. 0.]
轨道数目：4
粒子数目(1, 1)



In [3]:

from Adapt import AdaptVQE_Y
from qiskit_nature.second_q.operators import FermionicOp
a = AdaptVQE_Y(es_problem=problem)
#a.operator_pool

In [5]:
a.opertor_pool_2


[PauliSumOp(SparsePauliOp(['YYXY', 'XYYY', 'XXXY', 'YXYY', 'XYXX', 'YYYX', 'YXXX', 'XXYX'],
               coeffs=[-0.125+0.j, -0.125+0.j, -0.125+0.j,  0.125+0.j, -0.125+0.j,  0.125+0.j,
   0.125+0.j,  0.125+0.j]), coeff=1.0)]

In [24]:
from qiskit_nature.second_q.mappers import QubitConverter,JordanWignerMapper
op2 = FermionicOp({"-_0 +_0 +_1": 1}, num_spin_orbitals=5)
op = [op2]*2
mapper = JordanWignerMapper()
mapper.map(op2)
mapper.map(op[0])

PauliSumOp(SparsePauliOp(['IIIXZ', 'IIIYZ', 'IIIXI', 'IIIYI'],
              coeffs=[0.25+0.j  , 0.  -0.25j, 0.25+0.j  , 0.  -0.25j]), coeff=1.0)

In [ ]:
from qiskit_nature.second_q.circuit.library.ansatzes.utils import generate_fermionic_excitations
generate_fermionic_excitations(num_excitations=2,num_particles=problem.num_particles,num_spatial_orbitals=int(problem.num_spin_orbitals//2))


[((0, 2), (1, 3))]

In [ ]:
op2 = FermionicOp({"-_0 +_0 +_1": 1}, num_spin_orbitals=2)
op2

FermionicOp({'-_0 +_0 +_1': 1}, num_spin_orbitals=2, )

In [ ]:
mapper = JordanWignerMapper()
mapper.map(op2)

AttributeError: 'FermionicOp' object has no attribute 'to_list'